In [15]:
import pandas as pd
import math
import numpy as np
data = pd.read_csv('1KMinData_TXF.csv')

import pickle
XX = pickle.load(open("bias_P&V_15\2\3\5\20_X","rb"))
YY = pickle.load(open("bias_P&V_15\2\3\5\20_Y","rb"))

In [16]:
from sklearn.utils import shuffle
XX, YY = shuffle(XX, YY, random_state=1)

train_x = XX[:10000,:,:]
test_x = XX[10000:,:,:]
train_y = YY[:10000,:]
test_y = YY[10000:,:]

'''
import matplotlib.pyplot as plt
%matplotlib inline
plt.imshow(train_x[4643,:,:,0],cmap='gray')
plt.show()
'''

print(train_x.shape)
print(test_y.shape)

(10000, 20, 20)
(828, 3)


In [17]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [18]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID') #1x1 patch stride = 1

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], #Due to 2x2, half output size
                        strides=[1, 2, 2, 1], padding='VALID')

In [19]:
x = tf.placeholder(tf.float32, shape=[None, 20,20])
y_ = tf.placeholder(tf.float32, shape=[None, 3])

In [20]:
W_conv1 = weight_variable([5, 5, 1, 16]) #index 2: input size(depth = 5); index 3:output size(depth = 32)
b_conv1 = bias_variable([16])
x_image = tf.reshape(x, [-1,20,20,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

print(h_conv1)
print(h_pool1)

Tensor("Relu_3:0", shape=(?, 16, 16, 16), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 8, 8, 16), dtype=float32)


In [21]:
W_conv2 = weight_variable([3, 3, 16, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

print(h_conv2)
print(h_pool2)

Tensor("Relu_4:0", shape=(?, 6, 6, 64), dtype=float32)
Tensor("MaxPool_3:0", shape=(?, 3, 3, 64), dtype=float32)


In [22]:
W_fc1 = weight_variable([3 * 3 * 64, 128])
b_fc1 = bias_variable([128])

h_pool2_flat = tf.reshape(h_pool2, [-1, 3 * 3 * 64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([128, 3])
b_fc2 = bias_variable([3])

'''
logits = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_conv = tf.nn.softmax(logits)
'''

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

a = tf.matmul(h_fc1_drop, W_fc2)
b = b_fc2

In [23]:
print(x)
print(x_image)
print(h_conv1)
print(h_pool1)
print(h_conv2)
print(h_pool2)
print(h_pool2_flat)
print(h_fc1)
print(h_fc1_drop)
print(y_conv)

Tensor("Placeholder_3:0", shape=(?, 20, 20), dtype=float32)
Tensor("Reshape_2:0", shape=(?, 20, 20, 1), dtype=float32)
Tensor("Relu_3:0", shape=(?, 16, 16, 16), dtype=float32)
Tensor("MaxPool_2:0", shape=(?, 8, 8, 16), dtype=float32)
Tensor("Relu_4:0", shape=(?, 6, 6, 64), dtype=float32)
Tensor("MaxPool_3:0", shape=(?, 3, 3, 64), dtype=float32)
Tensor("Reshape_3:0", shape=(?, 576), dtype=float32)
Tensor("Relu_5:0", shape=(?, 128), dtype=float32)
Tensor("dropout_1/mul:0", shape=(?, 128), dtype=float32)
Tensor("Softmax_1:0", shape=(?, 3), dtype=float32)


In [24]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv+1e-50), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 
'''
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, y_)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))

train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) 

test= tf.log(y_conv)
'''


'\ncross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, y_)\ncorrect_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))\n\ntrain_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)\n\naccuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) \n\ntest= tf.log(y_conv)\n'

In [25]:
sess.run(tf.initialize_all_variables())
for i in range(1000):
    train_x, train_y = shuffle(train_x, train_y, random_state=2)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:train_x, y_: train_y, keep_prob: 0.5})

        print("step %d, training accuracy %g"%(i, train_accuracy))
        print("test accuracy %g"%accuracy.eval(feed_dict={
        x: test_x, y_: test_y, keep_prob: 0.5}))
        
        print("cross %g"%cross_entropy.eval(feed_dict={
        x: train_x, y_: train_y, keep_prob: 0.5}))
        
        #print(x_image.eval(feed_dict={x: train_x, y_: train_y, keep_prob: 0.5}))
        
        #print(W_conv1.eval)
        
        #print(h_conv1.eval(feed_dict={x: train_x, y_: train_y, keep_prob: 0.5}))
        
        
        
            

    train_step.run(feed_dict={x: train_x[:200], y_: train_y[:200], keep_prob: 0.1})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: test_x, y_: test_y, keep_prob: 0.5}))

ResourceExhaustedError: OOM when allocating tensor with shape[10000,16,16,16]
	 [[Node: Conv2D_2 = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](Reshape_2, Variable_8/read)]]

Caused by op 'Conv2D_2', defined at:
  File "/home/joufan/.conda/envs/tf_conda/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/joufan/.conda/envs/tf_conda/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/joufan/.conda/envs/tf_conda/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/joufan/.conda/envs/tf_conda/lib/python3.5/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/home/joufan/.conda/envs/tf_conda/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().star

In [ ]:
sess.close()